<a href="https://colab.research.google.com/github/baskayj/Boundary-behaviour-in-stochastic-differential-equations-used-in-Finance/blob/master/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
import json
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp

from collections import namedtuple
from datetime import datetime

mc_rep = 10000                                                                  #Number of generated trajectories
s_freq = 100                                                                    #Sampling frequency

In [0]:
#To load and save simulation data
!ls -l  #Making sure we are at the right directory
DATA_PATH = 'Data/'

total 1668
-rwxrwxrwx 1 baskayj baskayj  270676 Oct  5 12:22  Analyser.ipynb
drwxrwxrwx 1 baskayj baskayj    4096 Oct  5 12:36  Data
-rwxrwxrwx 1 baskayj baskayj   19449 Oct  5 12:36  Models.ipynb
-rwxrwxrwx 1 baskayj baskayj 1412037 Oct  4 09:12 'Old Code.ipynb'
-rwxrwxrwx 1 baskayj baskayj     144 Oct  4 09:12  start_colab.sh


In [0]:
#CIR MODEL
#There are multiple finite-diffrence schemes for simulation including,
#Euler-Maryama Scheme: X[i] = X[i-1] + kappa*(theta-X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt)
#Weighted Milstein Scheme: X[i] = (X[i-1] + kappa*(theta-alpha*X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt) + (1/4)*np.power(sigma,2)*dt*(np.power(W[i-1],2)-1))/(1+(1-alpha)*kappa*dt)
#To be continued...

#gamma = 2*kappa*theta/sigma^2

#-------------------------------------------------------------------------------
#Path(returns the entire path as a NumPy array)
#Euler-Maryama Scheme
def CIR_path(X0, dt, T, kappa, theta, sigma, seed):
  N = int(T/dt)
  np.random.seed(seed)
  W = np.random.normal(size=N)
  X = np.zeros(N)
  X[0] = X0
  for i in range(1,N):
    x = X[i] = X[i-1] + kappa*(theta-X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt)
    if x < 0 :
      X[i] = 0
    else:
      X[i] = x
  del W
  return X


#Weighted Milstein Scheme
def CIR_path_wms(X0, dt, T, kappa, theta, sigma, seed):
  N = int(T/dt)
  alpha = 0.8
  np.random.seed(seed)
  W = np.random.normal(size=N)
  X = np.zeros(N)
  X[0] = X0
  for i in range(1,N):
    x = X[i] = (X[i-1] + kappa*(theta-alpha*X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt) + (1/4)*np.power(sigma,2)*dt*(np.power(W[i-1],2)-1))/(1+(1-alpha)*kappa*dt)
    if x < 0 :
      X[i] = 0
    else:
      X[i] = x
  return X

#-------------------------------------------------------------------------------
#Process(Does not return entire trajectories)
#Euler-Maryama Scheme
def CIR_prcss(X0, dt, T, kappa, theta, sigma, seed):
  N = int(T/dt)
  np.random.seed(seed)
  W = np.random.normal(size=N)
  X = np.zeros(N)
  
  X_R = []         #Samples will be stored here
  tau_2X0 = T+dt   #Time for the trajectory to reach 2*X0 for the first time
  tau_0 = T+dt     #Time for the trajectory to reach 2*X0 for the first time
  R = []           #All the return values will be here
  

  X[0] = X0
  for i in range(1,N):
    x = X[i-1] + kappa*(theta-X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt)

    if x < 0 :
      if i*dt <= tau_0:
        tau_0 = i*dt
      X[i] = 0
    
    elif x > 2*X0:
      if i*dt <= tau_2X0:
        tau_2X0 = i*dt
      X[i] = x
    
    else:
      X[i] = x
    
    if (i%s_freq) == 0:
      X_R.append(X[i])
  X_R.append(X[N-1])
  R.append(X_R)
  R.append(tau_2X0)
  R.append(tau_0)
  
  del W
  del X
  del X_R
  
  return R

#Weighted Milstein Scheme
def CIR_prcss_wms(X0, dt, T, kappa, theta, sigma, seed):
  N = int(T/dt)
  alpha = 0.8
  np.random.seed(seed)
  W = np.random.normal(size=N)
  X = np.zeros(N)
  X[0] = X0
  for i in range(1,N):
    x = (X[i-1] + kappa*(theta-alpha*X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt) + (1/4)*np.power(sigma,2)*dt*(np.power(W[i-1],2)-1))/(1+(1-alpha)*kappa*dt)
    if x < 0 :
      X[i] = 0
    else:
      X[i] = x
  return X[N-1]

#Non-central chi^2 method
def CIR_prcss_nccs(X0, dt, T, kappa, theta, sigma, seed):
  k = ((4*kappa*theta)/(np.power(sigma,2)))
  N = int(T/dt)
  np.random.seed(seed)
  X = np.zeros(N)
  X[0] = X0
  for i in range(1,N):
    X[i] = np.random.noncentral_chisquare(k,X[i-1]*((4*kappa)/(np.power(sigma,2)))*np.exp(-kappa*dt)*(1-np.exp(-kappa*dt))) * np.exp(-kappa*dt) *(1/((4*kappa)/(np.power(sigma,2)))*np.exp(-kappa*dt)*(1-np.exp(-kappa*dt)))
  return X[N-1]

In [0]:
# CLASSES for MP

#CIR
class CIR():
  def __init__(self,X0,dt,T,kappa,theta,sigma):
    self.X0 = X0
    self.dt = dt
    self.T = T
    self.kappa = kappa
    self.theta = theta
    self.sigma = sigma
  def Prcss(self,seed):
    return CIR_prcss(self.X0, self.dt, self.T, self.kappa, self.theta, self.sigma, seed)

In [0]:
#MULTIPROCESS FUNCTION

def CIR_Prcss_mp(X0,dt,T,kappa,theta,sigma):
  cir = CIR(X0,dt,T,kappa,theta,sigma)
  pool = mp.Pool(processes=mp.cpu_count()-1)
  return pool.map(cir.Prcss,range(mc_rep))

In [0]:
#SAVING

def print_log(string):
  logs = open(f"{DATA_PATH}logs.txt","a")
  logs.write(string)
  logs.close()

def Save(model,name,var_name,P,var,R):

  with open(f"{DATA_PATH}{model}{name}_params.txt","w") as outfile1:
    json.dump(P._asdict(),outfile1)
  outfile1.close()

  for i in range(len(var_name)):
    with open(f"{DATA_PATH}{model}{name}_{var_name[i]}_values.txt","w") as outfile2:
      json.dump(var[i],outfile2)
    outfile2.close()
  for i in range(len(R)):
    np.save(f"{DATA_PATH}{model}{name}_{i}.npy",R[i])
  
  tvalue = datetime.fromtimestamp(int(time.time()))
  print_log(f"[{tvalue.strftime('%Y-%m-%d %H:%M:%S')}]Data saved to \"{DATA_PATH}\" succefully.\n")

------

# Relationship between the CIR-Process and the Noncentral $\chi ^2$-distribution

A noncentral $\chi^2$-distribution can be described with the probability density function:
$$f_\chi (x,k,\lambda) = \sum_{x = 0}^{\infty}{\frac{e^{-\lambda/2}(\lambda/2)^i}{i!}f_{Y_k+2i}(x)}$$
Where $Y_q$ is $\chi^2$ distributed with degrees of freedom of $q$.
The parameters of this thistribution:
* $k$ degrees of freedom
* $\lambda$ noncentrality parameter

According to [2] the CIR process can be simulated by setting the degrees of freedom and non-centrality parameters as follows:
* $k := \frac{4\kappa\theta}{\sigma^2} = 2\gamma$
* $\lambda := X_{t_n}\eta(dt)$
Where we introduced the function $\eta(dt)$ as 
$$ \eta(dt) := \frac{4\kappa}{\sigma^2} e^{-\kappa dt} (1-e^{-\kappa dt})$$

With this the $t_{n+1}$-th step looks like:
$$ X_{t_{n+1}} = \chi^2_k (\lambda) \frac{e^{-\kappa dt}}{\eta(dt)} $$

We can use **numpy.random.noncentral_chisquare** to draw random numbers from the noncentral $\chi^2$-distribution.


References:

[1] https://en.wikipedia.org/wiki/Noncentral_chi-squared_distribution

[2] https://arxiv.org/pdf/0802.4411.pdf

---

# EXPERIMENT: $T\rightarrow\infty$

In [0]:
#EXPERIMENT:

model = 'CIR_'

name = 'T_to infty'

tvalue = datetime.fromtimestamp(int(time.time()))
print_log(f"\n\n[{tvalue.strftime('%Y-%m-%d %H:%M:%S')}]Staring Experiment {model}{name}.\n")

#Parameters:
Parameters = namedtuple('Parameters','X0 dt T kappa theta sigma')
P = Parameters(X0 = 0.1, dt = (1/100), T = 100,kappa = 1, theta = 1, sigma = 0.6)

#Variables:
var_name = 'gamma'
var = [0.2,0.3,0.5,0.8,0.9,1,1.1,1.5,2,3,5,10]

#kappa = list(map(lambda p: p*np.power(P.sigma,2)*(1/(2*P.theta)),var))
#theta = list(map(lambda p: p*np.power(P.sigma,2)*(1/(2*P.kappa)),var))
sigma = list(map(lambda p: np.sqrt((2*P.kappa*P.theta)/p),var))

#Transforming the parameters into a more digestable form. This way the computation heavy part of the code needs less for cycles.
T = []
for i in range(len(var)):
  T.append([P.X0,P.dt,P.T,P.kappa,P.theta,sigma[i]])

#GENERATING ALL TRAJECTORIES
X = np.zeros((len(T),mc_rep,int(P.T/(P.dt*s_freq))))
tau_2X0 = np.zeros((len(T),mc_rep))
tau_0 = np.zeros((len(T),mc_rep))

for i in range(len(var)):
  start_time = time.time()
  print_log(f"Making trajectories for X0 = {T[i][0]},dt = {T[i][1]},T = {T[i][2]},kappa = {T[i][3]},theta = {T[i][4]},sigma = {T[i][5]}.\n")

  r = CIR_Prcss_mp(P.X0,P.dt,P.T,P.kappa,P.theta,sigma[i])

  for j in range(len(r)):
    X[i,j,:] = r[j][0]
    tau_2X0[i,j] = r[j][1]
    tau_0[i,j] = r[j][2]
  end_time = time.time()
  print_log(f"  Finished generating trajectories in {int((end_time-start_time)/60)} minutes and {end_time-start_time-int((end_time-start_time)/60)*60} seconds.[{i+1}/{len(T)}]\n")

#Saving the Data
var_name = [var_name]
var = [var]
R = [X,tau_2X0,tau_0]
del X
del tau_2X0
del tau_0
Save(model,name,var_name,P,var,R)

#Memory Management
del T
del R

tvalue = datetime.fromtimestamp(int(time.time()))
print_log(f"[{tvalue.strftime('%Y-%m-%d %H:%M:%S')}]The experiment finished.\n")

# EXPERIMENT: Using the Noncentral $\chi ^2$-distribution

In [0]:
#EXPERIMENT:Simulating the process using the noncentral chi^2-distribution

model = 'CIR_'

name = 'nccs_test'

print_log(f"\n\nStaring Experiment {model}{name}.\n")

#Parameters:
Parameters = namedtuple('Parameters','X0 dt T kappa theta sigma')
P = Parameters(X0 = 0.00001, dt = (1/10), T = 1,kappa = 1, theta = 1, sigma = 0.6)

#Variables:
var_name = 'gamma'
var = [0.1,0.5,0.9,1.1,1.5,2,5]

#kappa = list(map(lambda p: p*np.power(P.sigma,2)*(1/(2*P.theta)),var))
#theta = list(map(lambda p: p*np.power(P.sigma,2)*(1/(2*P.kappa)),var))
sigma = list(map(lambda p: np.sqrt((2*P.kappa*P.theta)/p),var))

#Transforming the parameters into a more digestable form. This way the computation heavy part of the code needs less for cycles.
T = []
for i in range(len(var)):
  T.append([P.X0,P.dt,P.T,P.kappa,P.theta,sigma[i]])

start_time = time.time()
#GENERATING ALL TRAJECTORIES
X = np.zeros((len(T),mc_rep))
for i in range(len(T)):
  print_log(f"Making trajectories for X0 = {T[i][0]},dt = {T[i][1]},T = {T[i][2]},kappa = {T[i][3]},theta = {T[i][4]},sigma = {T[i][5]}.\n")

  s_time = time.time()
  n = 0
  for x in CIR_Prcss_nccs_mp(T[i][0],T[i][1],T[i][2],T[i][3],T[i][4],T[i][5]):
    X[i,n] = x
    n += 1
  e_time = time.time()

  print_log(f"  Finished generating trajectories in {int((e_time-s_time)/60)} minutes and {e_time-s_time-int((e_time-s_time)/60)*60} seconds.[{i+1}/{len(T)}]\n")

#Saving the Data
var_name = [var_name]
var = [var]
Save(model,name,var_name,P,var,X)

#Memory Management
del T
del X

end_time = time.time()

print_log(f"The experiment finished in {int((end_time-start_time)/60)} minutes and {end_time-start_time-int((end_time-start_time)/60)*60} seconds.\n")

#EXPERIMENT: Finding the critical $\gamma$-value numerically

# MISC. TEST

In [0]:
#EXPERIMENT:

model = 'CIR_'

name = 'test'

print(f"\n\nStaring Experiment {model}{name}.\n")

#Parameters:
Parameters = namedtuple('Parameters','X0 dt T kappa theta sigma')
P = Parameters(X0 = 0.5, dt = (1/10), T = 1,kappa = 1, theta = 1, sigma = 0.6)

#Variables:
var_name = 'gamma'
var = [2]

#kappa = list(map(lambda p: p*np.power(P.sigma,2)*(1/(2*P.theta)),var))
#theta = list(map(lambda p: p*np.power(P.sigma,2)*(1/(2*P.kappa)),var))
sigma = list(map(lambda p: np.sqrt((2*P.kappa*P.theta)/p),var))

#Transforming the parameters into a more digestable form. This way the computation heavy part of the code needs less for cycles.
T = []
for i in range(len(var)):
  T.append([P.X0,P.dt,P.T,P.kappa,P.theta,sigma[i]])

start_time = time.time()
#GENERATING ALL TRAJECTORIES
X = np.zeros((len(T),mc_rep))
for i in range(len(T)):
  print(f"Making trajectories for X0 = {T[i][0]},dt = {T[i][1]},T = {T[i][2]},kappa = {T[i][3]},theta = {T[i][4]},sigma = {T[i][5]}.\n")

  s_time = time.time()
  n = 0
  for x in CIR_Prcss_mp(T[i][0],T[i][1],T[i][2],T[i][3],T[i][4],T[i][5]):
    X[i,n] = x
    n += 1
  e_time = time.time()

  print(f"  Finished generating trajectories in {int((e_time-s_time)/60)} minutes and {e_time-s_time-int((e_time-s_time)/60)*60} seconds.[{i+1}/{len(T)}]\n")

#Saving the Data
var_name = [var_name]
var = [var]
Save(model,name,var_name,P,var,X)

#Memory Management
del T
del X

end_time = time.time()

print(f"The experiment finished in {int((end_time-start_time)/60)} minutes and {end_time-start_time-int((end_time-start_time)/60)*60} seconds.\n")



Staring Experiment CIR_test.

Making trajectories for X0 = 0.5,dt = 0.1,T = 1,kappa = 1,theta = 1,sigma = 1.0.

  Finished generating trajectories in 0 minutes and 0.37604784965515137 seconds.[1/1]

The experiment finished in 0 minutes and 0.3887605667114258 seconds.



---

#### 10k x 1k grid runtimes, and cpu stress
* All Cores: 	22.3 sec 	100% utilizitaion
* 3 Cores:	26.3 sec 	80% utilizitaion
* 2 Cores:	39.0 sec	60 % utilizitaion
* 1 Core:		78.3 sec	30 % utilizitaion
* Colab:		52.8 sec